In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [75]:
input_text = au.read_txt_file_lines()

a = int(input_text[0].split(':')[1].lstrip())
b = int(input_text[1].split(':')[1].lstrip())
c = int(input_text[2].split(':')[1].lstrip())
print(a, b, c)

programme = [int(x) for x in input_text[4].split(':')[1].lstrip().split(',')]
print(programme)

23999685 0 0
[2, 4, 1, 1, 7, 5, 1, 5, 0, 3, 4, 4, 5, 5, 3, 0]


In [76]:
class Machine:
    def __init__(self, a, b, c): 
        self.a = a
        self.b = b
        self.c = c
        self.output = []

    def lit(self, op):
        return int(op)
    
    def combo(self, op):
        if op < 4:
            return int(op)
        elif op == 4:
            return self.a 
        elif op == 5:
            return self.b
        elif op == 6:
            return self.c 
        else:
            assert False, op

    def bit_xor(self, x, y):
        assert type(x) == int, x
        assert type(y) == int, y

        bin_x = bin(x)[2:]
        bin_y = bin(y)[2:]

        while len(bin_x) > len(bin_y):
            bin_y = '0' + bin_y
        
        while len(bin_x) < len(bin_y):
            bin_x = '0' + bin_x 

        new = []
        for ii, x_ in enumerate(bin_x):
            y_ = bin_y[ii]
            if int(x_) + int(y_) == 1:
                new.append('1')
            else:
                new.append('0')

        new = int(''.join(new), 2)
        return new
    
    def call(self, code, op):
        if code == 0:
            return self.adv(code, op)
        if code == 1:
            return self.bxl(code, op)
        if code == 2:
            return self.bst(code, op)
        if code == 3:
            return self.jnz(code, op)
        if code == 4:
            return self.bxc(code, op)
        if code == 5:
            return self.out(code, op)
        if code == 6:
            return self.bdv(code, op)
        if code == 7:
            return self.cdv(code, op)

    def adv(self, code, op):
        assert code == 0, code 
        div = self.a / (2 ** self.combo(op))
        div = int(np.floor(div))
        self.a = div 
        return None

    def bxl(self, code, op):
        assert code == 1, code 
        val = self.bit_xor(self.b, self.lit(op))
        self.b = val
        return None

    def bst(self, code, op):
        assert code == 2, code 
        val = self.combo(op) % 8
        self.b = val 
        return None

    def jnz(self, code, op):
        assert code == 3, code 
        if self.a == 0:
            return None 
        else:
            op = self.lit(op)
            if op != code:
                return op
            else:
                return None 
            
    def bxc(self, code, op):
        assert code == 4, code 
        val = self.bit_xor(self.b, self.c)
        self.b = val
        return None

    def out(self, code, op):
        assert code == 5, code 
        val = self.combo(op) % 8
        self.output.append(val)
        # print(val)
        return None

    def bdv(self, code, op):
        assert code == 6, code 
        div = self.a / (2 ** self.combo(op))
        div = int(np.floor(div))
        self.b = div 
        return None

    def cdv(self, code, op):
        assert code == 7, code 
        div = self.a / (2 ** self.combo(op))
        div = int(np.floor(div))
        self.c = div 
        return None



In [65]:
print(programme, a, b, c)
# programme = [0,3,5,4,3,0]
# a, b, c = 117440, 0, 0
m = Machine(a, b, c)

instruct = 0
while instruct < len(programme):
    tmp = m.call(programme[instruct], programme[instruct + 1])
    if tmp is None:
        instruct += 2
    else:
        instruct = tmp 

print(m.output)

print(','.join([str(x) for x in m.output]))

[0, 3, 5, 4, 3, 0] 2024 0 0
[0, 3, 5, 4, 3, 0]
0,3,5,4,3,0


## part 2

In [119]:
class Machinep2:
    def __init__(self, a, b, c): 
        self.a = a
        self.b = b
        self.c = c
        self.output = []
    
    def bit_xor(self, x, y):
        bin_x = bin(x)[2:]
        bin_y = bin(y)[2:]
        while len(bin_x) > len(bin_y):
            bin_y = '0' + bin_y
        while len(bin_x) < len(bin_y):
            bin_x = '0' + bin_x 

        new = []
        for ii, x_ in enumerate(bin_x):
            y_ = bin_y[ii]
            if int(x_) + int(y_) == 1:
                new.append('1')
            else:
                new.append('0')
        return int(''.join(new), 2)
    
    def call(self, code, op, it):
        print('it:', it, code, op, (self.a, self.b, self.c))
        if self.a == 0:
            print(self.a, self.b, self.c)
        if code == 0:
            return self.adv(code, op)
        if code == 1:
            return self.bxl(code, op)
        if code == 2:
            return self.bst(code, op)
        if code == 3:
            return self.jnz(code, op)
        if code == 4:
            return self.bxc(code, op)
        if code == 5:
            return self.out(code, op)
        if code == 6:
            assert False
        if code == 7:
            return self.cdv(code, op)
        
    def combo(self, op):
        if op < 4:
            return int(op)
        elif op == 4:
            return self.a 
        elif op == 5:
            return self.b

    def adv(self, code, op):  # 0
        div = self.a / (2 ** self.combo(op))
        self.a = int(np.floor(div)) 
        return None

    def bxl(self, code, op):   # 1
        self.b = self.bit_xor(self.b, op)
        return None

    def bst(self, code, op):  # 2
        self.b = self.combo(op) % 8 
        return None

    def jnz(self, code, op):  # 3
        if self.a != 0:
            return op 
        return None
            
    def bxc(self, code, op): # 4
        self.b = self.bit_xor(self.b, self.c)
        return None

    def out(self, code, op):  # 5
        self.output.append(self.combo(op) % 8)
        print('output!:', self.output, op, self.combo(op), self.a, self.b, self.c)
        return None

    def cdv(self, code, op):  # 7
        div = self.a / (2 ** self.combo(op))
        self.c  = int(np.floor(div))
        return None



In [128]:
# print(programme, a, b, c)
print(programme, a, b, c)
# programme = [0,3,5,4,3,0]
# a, b, c = 117440, 0, 0
a_init = 310096999859990
m = Machinep2(a_init, b, c)

instruct = 0
it = 0
while instruct < len(programme):
    tmp = m.call(programme[instruct], programme[instruct + 1], it=it)
    if tmp is None:
        instruct += 2
    else:
        instruct = tmp 

    it += 1
print(m.output)
print(m.a, m.b, m.c, it)
# print(','.join([str(x) for x in m.output]))

[2, 4, 1, 1, 7, 5, 1, 5, 0, 3, 4, 4, 5, 5, 3, 0] 23999685 0 0
it: 0 2 4 (310096999859990, 0, 0)
it: 1 1 1 (310096999859990, 6, 0)
it: 2 7 5 (310096999859990, 7, 0)
it: 3 1 5 (310096999859990, 7, 2422632811406)
it: 4 0 3 (310096999859990, 2, 2422632811406)
it: 5 4 4 (38762124982498, 2, 2422632811406)
it: 6 5 5 (38762124982498, 2422632811404, 2422632811406)
output!: [4] 5 2422632811404 38762124982498 2422632811404 2422632811406
it: 7 3 0 (38762124982498, 2422632811404, 2422632811406)
it: 8 2 4 (38762124982498, 2422632811404, 2422632811406)
it: 9 1 1 (38762124982498, 2, 2422632811406)
it: 10 7 5 (38762124982498, 3, 2422632811406)
it: 11 1 5 (38762124982498, 3, 4845265622812)
it: 12 0 3 (38762124982498, 6, 4845265622812)
it: 13 4 4 (4845265622812, 6, 4845265622812)
it: 14 5 5 (4845265622812, 4845265622810, 4845265622812)
output!: [4, 2] 5 4845265622810 4845265622812 4845265622810 4845265622812
it: 15 3 0 (4845265622812, 4845265622810, 4845265622812)
it: 16 2 4 (4845265622812, 4845265622810

In [121]:
1874976561404 % 8

4

In [73]:
programme = [0,3,5,4,3,0]
tuple(m.output) == tuple(programme)

True

In [80]:
tuple(m.output) == tuple(programme)

False